In [ ]:
# %pip install langchain
# %pip install chromadb
# %pip install pandas
# %pip install requests
# %pip install langchain-openai
# %pip install --upgrade chromadb
# %pip install langchain-community


   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------------- 2.4/2.4 MB 29.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [106]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.agents import AgentType
from langchain.agents import Tool, initialize_agent
import chromadb
from langchain.embeddings import OpenAIEmbeddings
import pandas as pd
import requests
import re
from collections import Counter
from datetime import datetime, timedelta
from chromadb import Client


In [109]:
api_key = ""
llm = ChatOpenAI(temperature=0.7, openai_api_key=api_key, model='gpt-4')
embeddings = OpenAIEmbeddings(openai_api_key=api_key)
chain =  llm

In [110]:
client = chromadb.PersistentClient(path=".")
doc_collection = client.get_or_create_collection(name="hidrogeno_verde")

In [111]:
def store_in_chromadb(documents, metadata_list):
    for idx, (doc, metadata) in enumerate(zip(documents, metadata_list)):
        embedding = embeddings.embed_query(doc)  # Genera el embedding
        doc_collection.add(
            ids=[str(idx)],  # ID único para cada documento
            documents=[doc],  # Documento en formato texto
            embeddings=[embedding],  # Embedding generado
            metadatas=[metadata]  # Información adicional (opcional)
        )


In [112]:
#Se puede omitir si ya tiene cargados los datos
df = pd.read_csv('hidrogeno_verde.csv')
df['date'] = pd.to_datetime(df['date'], format='%b %d, %Y @ %H:%M:%S.%f')
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df['combined'] = df['title'] + ' ' + df['text']
documents = df['text'].tolist() 
metadata_list = df.drop(columns=['text']).to_dict(orient='records') 

# Almacenar en ChromaDB
store_in_chromadb(documents, metadata_list)
print(metadata_list[:5])  # Muestra los primeros 5 metadatos


[{'date': '2024-10-19', 'id_news': '54.272.642', 'title': 'Ausenco impulsa la transición energética a través de servicios de consultoría especializados en hidrógeno verde', 'url': 'https://tierramarillano.cl/2024/10/20/ausenco-impulsa-la-transicion-energetica-a-traves-de-servicios-de-consultoria-especializados-en-hidrogeno-verde/', 'polarity_score': '-', 'polarity_label': '-', 'media_outlet': 'tierraamarillano', 'country': 'chile', 'combined': 'Ausenco impulsa la transición energética a través de servicios de consultoría especializados en hidrógeno verde Santiago, octubre de 2024 – Como parte de la celebración del Día del Hidrógeno Verde, organizada por H2 Chile, Ausenco refuerza su compromiso con un futuro energético más limpio y sostenible. Este evento, realizado en conmemoración del Día Internacional del Hidrógeno, reunió a líderes de la industria para discutir los avances, desafíos y oportunidades que presenta el hidrógeno verde en la descarbonización y el desarrollo sostenible. El

In [113]:
# Función para realizar la búsqueda en ChromaDB
def search_in_chromadb(query):
    # Realiza la búsqueda en ChromaDB utilizando la consulta
    results = doc_collection.query(
        query_embeddings=embeddings.embed_query(query),
        n_results=3  # Número de resultados a obtener
    )
    return results




In [1]:
def search_tool(query):
    search_results = search_in_chromadb(query)
    return "\n".join([str(doc) for doc in search_results['documents']])

def extract_dates_from_query(query):
    import re
    # Busca fechas en formato "dd-mm-yyyy" o "dd de mes de yyyy"
    match = re.findall(r"(\d{1,2})[-/\s]?(\w+)?[-/\s]?(\d{4})", query)
    if len(match) >= 2:
        start_date = "-".join(reversed(match[0]))
        end_date = "-".join(reversed(match[1]))
        return start_date, end_date
    return None, None

# def convert_metadata_dates_to_datetime(metadata):
#     # Convertir la fecha en cada metadato a datetime
#     for doc in metadata:
#         # Si la fecha existe y es una cadena, la convertimos a datetime
#         if 'date' in doc and isinstance(doc['date'], str):
#             try:
#                 doc['date'] = datetime.strptime(doc['date'], "%Y-%m-%d")
#             except ValueError:
#                 print(f"Fecha incorrecta en el documento {doc}. No se puede convertir.")
#     return metadata

def search_in_chromadb_with_dates(query, start_date, end_date):
    results = doc_collection.get()
    filtered_results = [
            doc for doc, metadata in zip(results['documents'], results['metadatas'])
            if 'date' in metadata and start_date <= metadata['date'] <= end_date
        ]
    return filtered_results

def chromadb_tool_with_dates(query):
    if not query.strip():
        return "La consulta está vacía. Por favor, proporciona una consulta válida."
    
    # Extraer fechas de la consulta
    start_date, end_date = extract_dates_from_query(query)
    if not start_date or not end_date:
        return "No se encontraron fechas válidas en la consulta. Por favor, incluye un rango de fechas claro."
    
    # Realizar la búsqueda
    try:
        search_results = search_in_chromadb_with_dates("personajes", start_date, end_date)
        # character_counts = count_character_mentions(search_results)
        return f"Personajes más mencionados entre {search_results}"
    except Exception as e:
        return f"Hubo un error al buscar en la base de datos: {str(e)}"



# Función para búsqueda en Wikipedia
def wikipedia_search(query):
    response = requests.get(f"https://es.wikipedia.org/w/api.php", params={
        "action": "query",
        "format": "json",
        "titles": query,
        "prop": "extracts",
        "exintro": True
    })
    pages = response.json()['query']['pages']
    page = next(iter(pages.values()))
    return page.get('extract', "No se encontró información en Wikipedia.")
# Crear herramientas
tools = [
     Tool(
        name="ChromaDB Search by Date",
        func=chromadb_tool_with_dates,
        description="Busca en ChromaDB documentos relevantes en un rango de fechas y cuenta menciones de personajes."
    ),
    Tool(
        name="ChromaDB Search",
        func=search_tool,
        description="Busca en la base de datos de ChromaDB los documentos relevantes."
    ),
    Tool(
        name="Wikipedia Search",
        func=wikipedia_search,
        description="Consulta Wikipedia para obtener más información sobre el tema."
    )
]

prompt_template = """
Eres un agente que puede interactuar con herramientas para obtener información relevante para responder preguntas. 
Tienes acceso a dos herramientas:

1. **ChromaDB Search**: Busca en una base de datos de documentos para obtener información relevante.
2. **Wikipedia Search**: Realiza una consulta a Wikipedia para obtener un resumen de un tema.
2. **ChromaDB Search by date**: Realiza una consulta en la Base de datos entre dos fechas específicas y obtener información relevante.


Cuando recibas una pregunta, debes decidir qué herramienta usar y cómo responder de manera concisa y precisa. 
Si es posible, utiliza primero la búsqueda en ChromaDB, si no encuentras suficiente información, consulta Wikipedia.
"""

# Crear el PromptTemplate
prompt = PromptTemplate(input_variables=["tools", "query", "agent_scratchpad", "tool_names"], template=prompt_template)
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)




NameError: name 'Tool' is not defined

In [167]:
# Ejecutar el agente con una consulta usando `invoke`
query = "¿Quiénes son los 5 personajes más mencionados entre el 16-09-2024 y el 16-09-2024?"
response = agent.invoke(query)

# Mostrar la respuesta
print(response)




> Entering new AgentExecutor chain...
Necesito buscar en la base de datos de ChromaDB utilizando la herramienta de búsqueda por fecha y contar las menciones de personajes.
Action: ChromaDB Search by Date
Action Input: 16-09-2024 to 16-09-2024
Observation: Personajes más mencionados entre ['Un cambio en la matriz energética con miras a convertirnos en un país carbono neutral y resiliente antes de 2050 es el principal objetivo del Plan de Acción Hidrógeno Verde 2023-2030, el cual busca aprovechar la extensa cadena de valor de este combustible limpio para posicionar a Chile como productor, consumidor y también como exportador, aprovechando además las potencialidades que brinda como ruta el Estrecho de Magallanes. Es en este contexto que la Pontificia Universidad Católica de Valparaíso (PUCV) se encuentra ejecutando un proyecto del Ministerio de Energía, en alianza con la Universidad de Magallanes (UMAG) y el Gobierno Regional, cuyo objetivo principal es elaborar un Plan de Infraestructu

Las 4 personas que aparecen en la noticia del 16-09-2024 son: Álvaro Peña, Alex Paz, Matías Valenzuela y Berta Vivar  
